# SVR model

## Import libraries and download the data

In [22]:
#Download libraries
import re
from collections import defaultdict
import pandas as pd
import altair as alt
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.svm import SVR
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [14]:
#Download the data
df = pd.read_csv('../data/train_data.zip')
df.head()

,external_id,month,year,monthly_number_of_sessions,monthly_unique_sessions,monthly_repeated_sessions,monthly_avg_length_of_session,monthly_avg_light_activity,monthly_avg_moderate_activity,monthly_avg_vigorous_activity,...,avg_wind_9_10,avg_wind_10_11,avg_wind_11_12,avg_wind_12_above,perfect_days,unacast_session_count,hpi,state_and_local_amount_per_capita,state_amount_per_capita,local_amount_per_capita
0,1900203,3,2019,0,0,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,78.0,323.61,0.132207,0.018519,0.113688
1,1900203,6,2018,0,0,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,4.0,111.0,323.61,0.132207,0.018519,0.113688
2,1900203,8,2018,0,0,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2.0,110.0,323.61,0.132207,0.018519,0.113688
3,MR00101775,1,2019,0,0,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,10.0,110.38,0.076247,0.011966,0.064281
4,MR00101775,8,2019,0,0,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,11.0,110.38,0.076247,0.011966,0.064281


## Test for the preprocessing function

In [135]:
#Categorical data
clean_df_2.loc[:, clean_df_2.dtypes == "object"].columns

Index(['external_id', 'state', 'income_class', 'density_class', 'climate'], dtype='object')

In [20]:
#Check that there are no missing values in the `Number_of_holidays` column
clean_df['Number_of_holidays'].isnull().sum()

0

In [22]:
#Check that every month has only one value for the `Number_of_holiday` column
number_of_error = 0
for month in range(12):
    for year in [2018, 2019]:
        df_jan_19 = clean_df[(clean_df['month'] == month+1) & (clean_df['year'] == year)]
        if len(df_jan_19['Number_of_holidays'].unique()) > 1:
            number_of_error += 1 
print(number_of_error)

0


In [26]:
#Check that the values in the `Green_2016` column for the states of North Carolina and Alaska are the right ones
for state in ['North Carolina', 'Alaska']:
    print(clean_df[clean_df['state'] == state]['Green_2016'].head(1))

60    12105.0
Name: Green_2016, dtype: float64
3121    5735.0
Name: Green_2016, dtype: float64


In [27]:
#Check that the values that were not NaNs in the `Green_2016` column remained the same
clean_df[['state', 'Green_2016']].iloc[3]

state         Mississippi
Green_2016       0.255869
Name: 3, dtype: object

In [29]:
df[['state', 'Green_2016']].iloc[3]

state         Mississippi
Green_2016       0.255869
Name: 3, dtype: object

In [109]:
for value in ['county', 'city', 'state', 'country', 'closest_place_category', 'income_class', 'density_class', 'climate', 'external_id']:
    print(df[value].isnull().sum()/df.shape[0])

0.0003990422984836393
0.05626496408619314
0.0
0.0
0.34916201117318435
0.0
0.0
0.07861133280127694
0.0


# SVR Model

## With raw data

In [ ]:
    #Use One Hot Encoder for the `climate` column
    #ohe = OneHotEncoder(sparse=False, dtype=int)
    #climate_df = pd.DataFrame(ohe.fit_transform(df_weather[['climate']]), columns=ohe.categories_[0])
    
       #output_data = pd.concat((output_data, climate_df), axis=1)
    #output_data.drop(columns = ['climate'], inplace=True)

In [9]:
#Kernel = ‘linear’
X_train, X_valid, y_train, y_valid = train_test_split(df.drop(columns=['unacast_session_count', 'county', 'city', 'state', 
                                                                       'country', 'closest_place_category', 'income_class',
                                                                      'density_class', 'climate', 'external_id']),
                                                    df[['unacast_session_count']],
                                                    test_size=0.15,
                                                    random_state=2020)

parameters = {'C':np.logspace(-5,5,40), 'gamma':np.logspace(-10,10,40)}
svr = SVR(kernel='linear')
svr_opt = RandomizedSearchCV(svr, parameters, cv=5, iid=False, n_iter=100)
svr_opt.fit(X_train, y_train.to_numpy().ravel())
print('The best parameter combination is {}.'.format(svc_opt.best_params_))
print('The accuracy on the test split is {:.2f}.'.format(svc_opt.score(X_test,y_test)))

C:\Users\sirin\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: could not convert string to float: "[{'question': 'accessible', 'avg_answer': 5.0}, {'question': 'regular', 'avg_answer': 1.0}, {'question': 'condition', 'avg_answer': 5.0}, {'question': 'revisit', 'avg_answer': 1.0}, {'question': 'safety', 'avg_answer': 5.0}, {'question': 'allages', 'avg_answer': 5.0}]"

  FitFailedWarning)
C:\Users\sirin\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: could not convert string to float: "[{'question': 'accessible', 'avg_answer': 5.0}, {'question': 'regular', 'avg_answer': 1.0}, {'question': 'condition', 'avg_answer': 5.0}, {'question': 'revisit', 'avg_answer': 1.0

KeyboardInterrupt: 

In [ ]:
# Kernel = ‘poly’
X_train, X_valid, y_train, y_valid = train_test_split(clean_df.drop(columns='unacast_session_count'),
                                                    clean_df[['unacast_session_count']],
                                                    test_size=0.15,
                                                    random_state=2020)

parameters = {'C':np.logspace(-5,5,40), 'gamma':np.logspace(-10,10,40)}
svr = SVR(kernel='poly')
svr_opt = RandomizedSearchCV(svr, parameters, cv=5, iid=False, n_iter=100)
svr_opt.fit(X_train, y_train.to_numpy().ravel())
print('The best parameter combination is {}.'.format(svc_opt.best_params_))
print('The accuracy on the test split is {:.2f}.'.format(svc_opt.score(X_test,y_test)))

In [ ]:
# Kernel = ‘rbf’

In [ ]:
# Kernel = ‘sigmoid’

In [ ]:
# Kernel = ‘precomputed’